# Libraries


In [174]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    col,
    substring,
    input_file_name,
    current_date,
    year,
    like,
    asc,
    desc,
    count,
    count_distinct,
    max,
    min,
    avg,
    array_agg,
    array_distinct,
    array_contains,
    row_number,
    rank,
    dense_rank,
    ntile,
    sum,
    lead,
    lag,
    cume_dist,
)


from pyspark.sql.types import (
    IntegerType,
    LongType,
    StructField,
    StructType,
    DateType,
    DoubleType,
    StringType,
    TimestampType,
)

# Spark Session


In [125]:
builder = SparkSession.builder.config(
    "spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"
).config(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Loading Data into spark


In [126]:
my_user_schema = StructType(
    [
        StructField("index", IntegerType(), nullable=False),
        StructField("organization_id", StringType(), nullable=False),
        StructField("name", StringType(), nullable=False),
        StructField("website", StringType(), nullable=False),
        StructField("country", StringType(), nullable=False),
        StructField("description", StringType(), nullable=False),
        StructField("founded", IntegerType(), nullable=False),
        StructField("industry", StringType(), nullable=False),
        StructField("employee_no", IntegerType(), nullable=False),
    ]
)

users_df = (
    spark.read.option("header", True)
    .schema(my_user_schema)
    .csv("./datasets/organizations-2000000.csv")
)


click_data_schema = StructType(
    [
        StructField("session_id", IntegerType(), nullable=False),
        StructField("IPID", IntegerType(), nullable=False),
        StructField("timestamp", TimestampType(), nullable=False),
        StructField("VHOST", StringType(), nullable=False),
        StructField("URL_FILE", StringType(), nullable=False),
        StructField("PAGE_NAME", StringType(), nullable=False),
        StructField("REF_URL_category", StringType(), nullable=False),
        StructField("page_load_error", IntegerType(), nullable=False),
        StructField("page_action_detail", StringType(), nullable=False),
        StructField("tip", StringType(), nullable=False),
        StructField("service_detail", StringType(), nullable=False),
        StructField("xps_info", StringType(), nullable=False),
        StructField("page_action_detail_EN", StringType(), nullable=False),
        StructField("service_detail_EN", StringType(), nullable=False),
        StructField("tip_EN", StringType(), nullable=False),
    ]
)

click_data_df = (
    spark.read.option("header", True)
    .option("delimiter", ";")
    .schema(click_data_schema)
    .csv("./datasets/BPI2016_Clicks_NOT_Logged_In.csv")
)

# Converting dataframes to delta tables


In [127]:
# users_df.write.mode("overwrite").format("delta").save("./output/users_df_delta")
# click_data_df.write.mode("overwrite").format("delta").save(
#     "./output/click_data_df_delta"
# )

# Querying delta tables


In [128]:
spark.sql(
    """
DESCRIBE HISTORY delta.`D:\\development\\learn_spark\\output\\users_df_delta`;
"""
).show()


spark.sql(
    """
DESCRIBE HISTORY delta.`D:\\development\\learn_spark\\output\\click_data_df_delta`;
"""
).show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      1|2024-01-17 14:01:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          0|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/3.5....|
|      0|2024-01-17 13:54:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|     Serializable|        false|{numFiles -> 8, n...|        NULL|Apache-Spark/3.5.

# Performing EDA on data


In [129]:
spark.read.format("delta").load("./output/click_data_df_delta/").printSchema()
spark.read.format("delta").load("./output/click_data_df_delta/").show(5)

spark.read.format("delta").load("./output/users_df_delta/").printSchema()
spark.read.format("delta").load("./output/users_df_delta/").show(5)

root
 |-- session_id: integer (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- VHOST: string (nullable = true)
 |-- URL_FILE: string (nullable = true)
 |-- PAGE_NAME: string (nullable = true)
 |-- REF_URL_category: string (nullable = true)
 |-- page_load_error: integer (nullable = true)
 |-- page_action_detail: string (nullable = true)
 |-- tip: string (nullable = true)
 |-- service_detail: string (nullable = true)
 |-- xps_info: string (nullable = true)
 |-- page_action_detail_EN: string (nullable = true)
 |-- service_detail_EN: string (nullable = true)
 |-- tip_EN: string (nullable = true)

+----------+-------+--------------------+-------------+--------------------+------------------+----------------+---------------+------------------+----+--------------+--------------------+---------------------+-----------------+------+
|session_id|   IPID|           timestamp|        VHOST|            URL_FILE|         PAGE_NAME|REF_URL_catego

## Optimizations

1. In first dataset it is a good choice to z-order the dataset by page name so the records corresponding to the same page are co-located
2. In the second dataset the same logic as point 1 is there for the country attribute.


In [130]:
# spark.sql(
#     """
# OPTIMIZE delta.`D:\\development\\learn_spark\\output\\users_df_delta` ZORDER BY (country);
# """
# ).show()
# spark.sql(
#     """
# OPTIMIZE delta.`D:\\development\\learn_spark\\output\\click_data_df_delta` ZORDER BY (PAGE_NAME);
# """
# ).show()

In [131]:
spark.sql(
    """
DESCRIBE HISTORY delta.`D:\\development\\learn_spark\\output\\users_df_delta`;
"""
).show(truncate=False)


spark.sql(
    """
DESCRIBE HISTORY delta.`D:\\development\\learn_spark\\output\\click_data_df_delta`;
"""
).show(truncate=False)

+-------+-----------------------+------+--------+---------+------------------------------------------+----+--------+---------+-----------+-----------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                       |job |notebook|clusterId|readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                                                                                                                                                                                                                      |userMetadata|engineInfo                         |
+-------+-----------------------+-

# Loading delta tables in spark dataframe


In [132]:
user_delta_table_v1 = (
    spark.read.format("delta").option("versionAsOf", 0).load("./output/users_df_delta/")
)
user_delta_table_v2 = (
    spark.read.format("delta").option("versionAsOf", 1).load("./output/users_df_delta/")
)

In [133]:
user_delta_table_v2.createOrReplaceTempView("user_delta_table_v2_tv")

In [134]:
user_delta_table_v2.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------
 index           | 1                             
 organization_id | 391dAA77fea9EC1               
 name            | Daniel-Mcmahon                
 website         | https://stuart-rios.biz/      
 country         | Cambodia                      
 description     | Focused eco-centric help-desk 
 founded         | 2013                          
 industry        | Sports                        
 employee_no     | 1878                          
-RECORD 1----------------------------------------
 index           | 2                             
 organization_id | 9FcCA4A23e6BcfA               
 name            | Mcdowell, Tate and Murray     
 website         | http://jacobs.biz/            
 country         | Guyana                        
 description     | Front-line real-time portal   
 founded         | 2018                          
 industry        | Legal Services                
 employee_no     | 9743                          


# Perfrom Group By and aggregations


In [135]:
# spark.sql(
#     """
#     SELECT
#     name,
#     count(distinct country) as distinct_country_count,
#     count(distinct employee_no) as distinct_no_of_employees
#     from user_delta_table_v2_tv
#     group by name
#     order by distinct_no_of_employees desc;
#     """
# ).show(5)


spark.sql(
    """
    SELECT
    
    employee_no,
    count(employee_no) as employee_count,
    array_distinct(array_agg(country)) as country_list,
    array_contains(array_agg(country), 'India') as contains_india

    from user_delta_table_v2_tv 
    group by employee_no
    order by employee_count desc;
    """
).show(5)


# spark.sql(
#     """
#     SELECT * from user_delta_table_v2_tv where name like '%Bradley PLC%';
#     """
# ).show(5)

+-----------+--------------+--------------------+--------------+
|employee_no|employee_count|        country_list|contains_india|
+-----------+--------------+--------------------+--------------+
|       4846|           261|[Samoa, Malta, Ja...|          true|
|       2384|           251|[Slovakia (Slovak...|          true|
|       3390|           250|[Rwanda, Uruguay,...|         false|
|       1589|           249|[Iran, Puerto Ric...|          true|
|       1961|           247|[Macedonia, Malay...|         false|
+-----------+--------------+--------------------+--------------+
only showing top 5 rows



In [136]:
user_delta_table_v2.groupBy("employee_no").agg(
    count(user_delta_table_v2.employee_no).alias("employee_no_count"),
    array_distinct(array_agg(user_delta_table_v2.country)).alias(
        "distinct_country_list"
    ),
    array_contains(array_agg(user_delta_table_v2.country), "India").alias(
        "is_india_a_part"
    ),
    avg(user_delta_table_v2.employee_no).alias("average_employees"),
).sort(desc("employee_no_count")).show(5)

+-----------+-----------------+---------------------+---------------+-----------------+
|employee_no|employee_no_count|distinct_country_list|is_india_a_part|average_employees|
+-----------+-----------------+---------------------+---------------+-----------------+
|       4846|              261| [Samoa, Malta, Ja...|           true|           4846.0|
|       2384|              251| [Slovakia (Slovak...|           true|           2384.0|
|       3390|              250| [Rwanda, Uruguay,...|          false|           3390.0|
|       1589|              249| [Iran, Puerto Ric...|           true|           1589.0|
|       7828|              247| [Saint Vincent an...|           true|           7828.0|
+-----------+-----------------+---------------------+---------------+-----------------+
only showing top 5 rows



# Windowing on data


In [163]:
ds_salaries_df = spark.read.options(header=True, inferSchema=True).csv(
    "./datasets/ds_salaries.csv"
)
ds_salaries_df.createOrReplaceTempView("ds_salaries_tv")

## Available functions

- Simple aggregation functions: avg, sum, min, max, count
- Row-wise ordering and ranking functions: row_number, rank, dense_rank, percent_rank, ntile (divides data into n buckets)
- Creating lagged columns: lag, lead
- Combining Windows and Calling Functions: over
- Analytic functions: cume_dist, first_value, last_value, nth_value
- Aggregate functions: collect_list, collect_set, corr, covar_pop, covar_samp, stddev, stddev_pop, stddev_samp, variance, var_pop, var_samp


In [197]:
spark.sql(
    """
select 
tab.*,
ROW_NUMBER() OVER(PARTITION BY job_title ORDER BY salary) as row_no,
RANK() OVER(PARTITION BY job_title ORDER BY salary) as rank,
dense_rank() OVER(PARTITION BY job_title ORDER BY salary) as dense_rank,
PERCENT_RANK() OVER(PARTITION BY job_title ORDER BY salary) as percent_rank,
NTILE(3) OVER(PARTITION BY job_title ORDER BY salary) as ntile,
lead(salary, 1, 404) OVER(PARTITION BY job_title ORDER BY salary) as lead,
lag(salary, 1, 404) OVER(PARTITION BY job_title ORDER BY salary) as lag,

avg(salary) OVER(PARTITION BY job_title ORDER BY salary) as avg,
min(salary) OVER(PARTITION BY job_title ORDER BY salary) as min,
max(salary) OVER(PARTITION BY job_title ORDER BY salary) as max,
sum(salary) OVER(PARTITION BY job_title ORDER BY salary) as sum,
cume_dist() OVER(PARTITION BY job_title ORDER BY salary) as cumdist
from ds_salaries_tv as tab;
"""
).show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+------+----+----------+------------------+-----+------+-----+------------------+------+------+------+-------------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|row_no|rank|dense_rank|      percent_rank|ntile|  lead|  lag|               avg|   min|   max|   sum|            cumdist|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+------+----+----------+------------------+-----+------+-----+------------------+------+------+------+-------------------+
| 77|     2021|              MI|             PT|3D Computer Visio...|400000|            INR|         5409|                IN|     

In [182]:
windowSpec = Window.partitionBy("job_title").orderBy("salary")


ds_salaries_df.withColumn("row_number", row_number().over(windowSpec)).withColumn(
    "rank", rank().over(windowSpec)
).withColumn("dense_rank", dense_rank().over(windowSpec)).withColumn(
    "avg", avg(col("salary")).over(windowSpec)
).withColumn(
    "min", min(col("salary")).over(windowSpec)
).withColumn(
    "max", max(col("salary")).over(windowSpec)
).withColumn(
    "sum", sum(col("salary")).over(windowSpec)
).withColumn(
    "lead", lead("salary", 2).over(windowSpec)
).withColumn(
    "lag", lag("salary", 2).over(windowSpec)
).withColumn(
    "cume_dist", cume_dist().over(windowSpec)
).show(
    50
)

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+----------+----+----------+------------------+------+--------+--------+--------+------+-------------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|row_number|rank|dense_rank|               avg|   min|     max|     sum|    lead|   lag|          cume_dist|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+----------+----+----------+------------------+------+--------+--------+--------+------+-------------------+
| 77|     2021|              MI|             PT|3D Computer Visio...|  400000|            INR|         5409|                IN|          50|              IN|         